In [1]:
import backtrader as bt
import datetime
#import matplotlib.pyplot as plt
import backtrader.indicators as btind


## Analyzers
They are objects that you load into cerebro that monitor your strategy as it runs. After cerebro has finished running the analyzers can be accessed through strategy objects that are returned by cerebro after running.

## Sizers
Sizers are classes that can be loaded into cerebro that are used to decide how many shares, units, contracts etc to buy or sell whenever self.buy() or self.sell() is called. Sizers will only be used for a calculation when no size is given. In other words, if your script contains a buy call like self.buy(size=100), the sizer will not be called. However if you call just self.buy(), cerebro will ask the sizer for the size to buy.

## Multiple DataFeed
https://www.backtrader.com/blog/posts/2017-04-09-multi-example/multi-example.html
https://backtest-rookies.com/2017/08/22/backtrader-multiple-data-feeds-indicators/
Adding the data. It is done in exactly the same way you add data for a single instrument. You just create the data object, feed it into cerebro, rinse and repeat. In the next() you can loop over the datafeeds one by one as:

for i, d in enumerate(self.datas)

In [ ]:
class TestStrategy1(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

In [ ]:
# Create a Stratey
class TestStrategy2(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        # Check if we are in the market
        if not self.position:
            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close
                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close
                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])
                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:
            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)

data = bt.feeds.GenericCSVData(
        dataname='/home/simone/Desktop/NLFF/indexes/indexesAAPL.csv',
        #Column containing datetime, open, close...
        datetime=1,open=2,high=3,low=4,close=5,volume=6,openinterest=-1,
        # Do not pass values before this date
        fromdate=datetime.datetime(2017, 5, 22),
        # Do not pass values after this date
        todate=datetime.datetime(2018, 6, 21),
        reverse=False)

# Add the Data Feed to Cerebro
cerebro.adddata(data)
# Add a strategy
cerebro.addstrategy(TestStrategy2)
# Add commission
cerebro.broker.setcommission(commission=0.001)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
class TestStrategy3(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else: 
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        if self.order:
            return
        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
        else:
            if self.dataclose[0] < self.sma[0]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

In [ ]:
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)

data = bt.feeds.GenericCSVData(
        dataname='/home/simone/Desktop/NLFF/indexes/indexesAAPL.csv',
        datetime=1,open=2,high=3,low=4,close=5,volume=6,openinterest=-1,
        fromdate=datetime.datetime(2017, 5, 22),
        todate=datetime.datetime(2018, 6, 21),
        reverse=False)

cerebro.adddata(data)
cerebro.addstrategy(TestStrategy3)
cerebro.broker.setcommission(commission=0.001)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
cerebro.plot()

In [10]:
class TestStrategy4(bt.Strategy):
    params = (
        ('maperiod', 15),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.trend_window = 15
        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %(order.executed.price,order.executed.value,order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else: 
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %(order.executed.price,order.executed.value,order.executed.comm))
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        if self.dataclose[0] > self.sma[0]:
#              buy(data=None, size=None, price=None, plimit=None, exectype=None, valid=None, tradeid=0, oco=None,
#                  trailamount=None, trailpercent=None, parent=None, transmit=True, **kwargs)
            self.order = self.buy(size=10)
            #self.order = self.sell(size=10, data = self.datas[self.trend_window])

In [11]:
class SentimentIndicator(bt.Indicator):
    lines = ('sentiment',)
    
    def next(self):
        date = self.data(0)
        return pdDatafrane[date]['prediction']

In [12]:
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)

data = bt.feeds.GenericCSVData(
        dataname='/home/simone/Desktop/NLFF/indexes/indexesAAPL.csv',
        datetime=1,open=2,high=3,low=4,close=5,volume=6,openinterest=-1,
        fromdate=datetime.datetime(2017, 5, 22),
        todate=datetime.datetime(2018, 6, 21),
        reverse=False)

cerebro.adddata(data)
cerebro.addstrategy(TestStrategy4)
cerebro.broker.setcommission(commission=0.001)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2017-05-25, Close, 154.31
2017-05-25, Close, 154.16
2017-05-25, Close, 154.15
2017-05-25, Close, 153.87
2017-05-26, SELL EXECUTED, Price: 154.00, Cost: -1540.00, Comm 1.54
2017-05-26, SELL EXECUTED, Price: 154.00, Cost: -1540.00, Comm 1.54
2017-05-26, SELL EXECUTED, Price: 154.00, Cost: -1540.00, Comm 1.54
2017-05-26, SELL EXECUTED, Price: 154.00, Cost: -1540.00, Comm 1.54
2017-05-26, Close, 153.90
2017-05-26, Close, 153.66
2017-05-26, Close, 153.47
2017-05-26, Close, 153.59
2017-05-26, Close, 153.75
2017-05-26, Close, 153.77
2017-05-26, Close, 153.61
2017-05-30, SELL EXECUTED, Price: 153.42, Cost: -1534.20, Comm 1.53
2017-05-30, SELL EXECUTED, Price: 153.42, Cost: -1534.20, Comm 1.53
2017-05-30, SELL EXECUTED, Price: 153.42, Cost: -1534.20, Comm 1.53
2017-05-30, SELL EXECUTED, Price: 153.42, Cost: -1534.20, Comm 1.53
2017-05-30, Close, 153.85
2017-05-30, Close, 153.69
2017-05-30, Close, 153.89
2017-05-30, Close, 154.36
2017-05-30, Close, 154.10
2017

2017-10-18, Close, 159.79
2017-10-18, Close, 160.37
2017-10-18, Close, 160.32
2017-10-18, Close, 160.06
2017-10-18, Close, 159.76
2017-10-19, SELL EXECUTED, Price: 156.75, Cost: -1567.50, Comm 1.57
2017-10-19, SELL EXECUTED, Price: 156.75, Cost: -1567.50, Comm 1.57
2017-10-19, SELL EXECUTED, Price: 156.75, Cost: -1567.50, Comm 1.57
2017-10-19, SELL EXECUTED, Price: 156.75, Cost: -1567.50, Comm 1.57
2017-10-19, SELL EXECUTED, Price: 156.75, Cost: -1567.50, Comm 1.57
2017-10-19, Close, 155.90
2017-10-19, Close, 155.95
2017-10-19, Close, 155.41
2017-10-19, Close, 155.07
2017-10-19, Close, 155.47
2017-10-19, Close, 155.49
2017-10-19, Close, 155.98
2017-10-20, Close, 156.83
2017-10-20, Close, 157.32
2017-10-20, Close, 157.47
2017-10-20, Close, 157.44
2017-10-20, Close, 157.20
2017-10-20, Close, 156.80
2017-10-20, Close, 156.16
2017-10-23, SELL EXECUTED, Price: 156.89, Cost: -1568.90, Comm 1.57
2017-10-23, SELL EXECUTED, Price: 156.89, Cost: -1568.90, Comm 1.57
2017-10-23, SELL EXECUTED, Pri

2017-12-28, Close, 170.84
2017-12-28, Close, 170.53
2017-12-28, Close, 171.08
2017-12-29, SELL EXECUTED, Price: 170.52, Cost: -1705.20, Comm 1.71
2017-12-29, SELL EXECUTED, Price: 170.52, Cost: -1705.20, Comm 1.71
2017-12-29, SELL EXECUTED, Price: 170.52, Cost: -1705.20, Comm 1.71
2017-12-29, SELL EXECUTED, Price: 170.52, Cost: -1705.20, Comm 1.71
2017-12-29, SELL EXECUTED, Price: 170.52, Cost: -1705.20, Comm 1.71
2017-12-29, SELL EXECUTED, Price: 170.52, Cost: -1705.20, Comm 1.71
2017-12-29, Close, 169.91
2017-12-29, Close, 169.64
2017-12-29, Close, 170.08
2017-12-29, Close, 170.12
2017-12-29, Close, 170.03
2017-12-29, Close, 169.98
2017-12-29, Close, 169.23
2018-01-02, Close, 170.97
2018-01-02, Close, 171.50
2018-01-02, Close, 171.21
2018-01-02, Close, 171.29
2018-01-02, Close, 171.69
2018-01-02, Close, 171.99
2018-01-02, Close, 172.26
2018-01-03, SELL EXECUTED, Price: 172.53, Cost: -1725.30, Comm 1.73
2018-01-03, SELL EXECUTED, Price: 172.53, Cost: -1725.30, Comm 1.73
2018-01-03, SE

2018-04-12, SELL EXECUTED, Price: 173.41, Cost: -1734.10, Comm 1.73
2018-04-12, SELL EXECUTED, Price: 173.41, Cost: -1734.10, Comm 1.73
2018-04-12, Close, 174.46
2018-04-12, Close, 174.34
2018-04-12, Close, 174.54
2018-04-12, Close, 174.82
2018-04-12, Close, 174.04
2018-04-12, Close, 174.75
2018-04-12, Close, 174.14
2018-04-13, SELL EXECUTED, Price: 174.78, Cost: -1747.80, Comm 1.75
2018-04-13, SELL EXECUTED, Price: 174.78, Cost: -1747.80, Comm 1.75
2018-04-13, SELL EXECUTED, Price: 174.78, Cost: -1747.80, Comm 1.75
2018-04-13, SELL EXECUTED, Price: 174.78, Cost: -1747.80, Comm 1.75
2018-04-13, SELL EXECUTED, Price: 174.78, Cost: -1747.80, Comm 1.75
2018-04-13, SELL EXECUTED, Price: 174.78, Cost: -1747.80, Comm 1.75
2018-04-13, SELL EXECUTED, Price: 174.78, Cost: -1747.80, Comm 1.75
2018-04-13, Close, 175.23
2018-04-13, Close, 174.57
2018-04-13, Close, 175.20
2018-04-13, Close, 175.11
2018-04-13, Close, 175.42
2018-04-13, Close, 174.78
2018-04-13, Close, 174.73
2018-04-16, SELL EXECUTE

In [ ]:
tickers=['AAPL','AMZN','GOOGL','MSFT','FB','INTC','CSCO','CMCSA','NVDA','NFLX']

cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)

for ticker in tickers:
    data = bt.feeds.GenericCSVData(
            dataname='/home/simone/Desktop/NLFF/indexes/indexes'+ticker+'.csv',
            
            datetime=1,open=2,high=3,low=4,close=5,volume=6,openinterest=-1,
            fromdate=datetime.datetime(2017, 5, 22),
            todate=datetime.datetime(2018, 6, 21),
            reverse=False)

cerebro.adddata(data)
cerebro.addstrategy(TestStrategy4)
cerebro.broker.setcommission(commission=0.001)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())